# Simple Inference with FCNv2

## Setup

In [1]:
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt

from scipy.signal import periodogram

# Set number of GPUs to use to 1
os.environ["WORLD_SIZE"] = "1"
# Set model registry as a local folder
model_registry = os.path.join(os.path.dirname(os.path.realpath(os.getcwd())), "models")
os.makedirs(model_registry, exist_ok=True)
os.environ["MODEL_REGISTRY"] = model_registry

# With the enviroment variables set now we import Earth-2 MIP
from earth2mip import registry, inference_ensemble
from earth2mip.initial_conditions import cds
from earth2mip.networks.fcnv2_sm import load as fcnv2_sm_load

## Load Model from registry

In [ ]:
# Load model(s) from registry
package = registry.get_model("fcnv2_sm")
print("loading FCNv2 small model, this can take a bit")
sfno_inference_model = fcnv2_sm_load(package)

## Get Input-Data from cds and run Inference

In [ ]:
print(f"{type(sfno_inference_model.in_channel_names)}: {sfno_inference_model.in_channel_names}")

In [ ]:
data_source = cds.DataSource(sfno_inference_model.in_channel_names)
output = "path/"
time = datetime.datetime(2018, 1, 1)
ds = inference_ensemble.run_basic_inference(
    sfno_inference_model,
    n=1,
    data_source=data_source,
    time=time,
)

In [ ]:
ds

In [ ]:
arr = ds.sel(channel="u200").values
f, pw = periodogram(arr, axis=-1, fs=1)
pw = pw.mean(axis=(1, 2))

l = ds.time - ds.time[0]  # noqa
days = l / (ds.time[-1] - ds.time[0])
cm = plt.cm.get_cmap("viridis")
for k in range(ds.sizes["time"]):
    day = (ds.time[k] - ds.time[0]) / np.timedelta64(1, "D")
    day = day.item()
    plt.loglog(f, pw[k], color=cm(days[k]), label=day)
plt.legend()
plt.ylim(bottom=1e-8)
plt.grid()
plt.savefig("u200_spectra.png")

# %%
day = (ds.time - ds.time[0]) / np.timedelta64(1, "D")
plt.semilogy(day, pw[:, 100:].mean(-1), "o-")
plt.savefig("u200_high_wave.png")

In [ ]:
model.in_channel_names

# Run Full Inference

In [2]:
from earth2mip.schema import EnsembleRun
from earth2mip.inference_ensemble import run_inference
import torch
import numpy as np
import earth2mip.initial_conditions

# Load model(s) from registry
package = registry.get_model("fcnv2_sm")
print("loading FCNv2 small model, this can take a bit")
model = fcnv2_sm_load(package)
perturb = None  # Use default perturbation if not specified
group = None  # Use default torch distributed group if not specified
progress = True  # Show progress bar


loading FCNv2 small model, this can take a bit


In [3]:
from earth2mip.weather_events import WeatherEvent, WeatherEventProperties, Domain, Window, Diagnostic

# Define diagnostics
diagnostic = Diagnostic(type="raw", channels=["u100", "u200"], nbins=10)
data_source = cds.DataSource(model.in_channel_names)

# Define domains
window = Window(
    name="global",
    lat_min=-90,
    lat_max=90,
    lon_min=0,
    lon_max=360,
    diagnostics=[diagnostic]
)

# Define weather event properties
weather_event_properties = WeatherEventProperties(
    name="example_event",
    start_time=datetime.datetime(2023, 5, 21),
    initial_condition_source='era5',
)

# Create WeatherEvent
weather_event = WeatherEvent(
    properties=weather_event_properties,
    domains=[window]
)

# EnsembleRun contains settings for the ensemble forecast
config = EnsembleRun(
    ensemble_members=3,  # Original number of ensemble members
    weather_model = "FCNv2",
    simulation_length = 2,
    weather_event = weather_event,
    output_dir = "inference_outputs"
    # Initialize other necessary parameters for EnsembleRun
)

In [4]:
# Step 3: Call the run_inference function with the modified config
run_inference(
    model=model,
    config=config,
    perturb=perturb,
    group=group,
    progress=progress,
    data_source=data_source
)

KeyboardInterrupt: 